In [ ]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
 
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
 
 
import datetime
import json
import os
import pprint
import random
import string
import sys
import tensorflow as tf

In [ ]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

In [ ]:
import numpy as np
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential
from keras.layers.advanced_activations import LeakyReLU
from keras.datasets import cifar10
(train_features, train_labels), (test_features, test_labels) = cifar10.load_data()
model = Sequential()

model.add(Conv2D(filters=16, kernel_size=(2, 2), padding="same", activation="relu", input_shape=(train_features.shape[1:])))
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))

model.add(Conv2D(filters=32, kernel_size=(3, 3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))

model.add(Conv2D(filters=64, kernel_size=(4, 4), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))

model.add(Flatten())

model.add(Dense(25600, activation="relu"))
model.add(Dense(25600, activation="relu"))
model.add(Dense(25600, activation="relu"))
model.add(Dense(25600, activation="relu"))
model.add(Dense(10, activation="softmax"))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(train_features, train_labels, validation_split=0.2, epochs=10, batch_size=128, verbose=1)

In [ ]:
!ps ax | grep python

In [ ]:
!nvidia-smi

In [ ]:
!kill -9 -1

In [ ]:
# Load the Drive helper and mount
from google.colab import drive
# This will prompt for authorization.
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys

!test -d SRGAN || git clone https://github.com/leftthomas/SRGAN
if not 'SRGAN' in sys.path:
  sys.path += ['SRGAN']
%cd SRGAN/
!pwd  

In [ ]:
import sys

!test -d SRGAN_Test || git clone https://github.com/goldenbili/SRGAN_Test.git
if not 'SRGAN_Test' in sys.path:
  sys.path += ['SRGAN_Test']
%cd SRGAN_Test/
!pwd  

In [ ]:
!wget http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_HR.zip
!wget http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_valid_HR.zip
!unzip -q DIV2K_train_HR.zip -d data
!unzip -q DIV2K_valid_HR.zip -d data
!rm DIV2K_train_HR.zip
!rm DIV2K_valid_HR.zip

In [ ]:
!cp ../drive/My\ Drive/SRGAN/DIV2K_train_HR.zip ./data/
!cp ../drive/My\ Drive/SRGAN/DIV2K_valid_HR.zip ./data/
!unzip -q ./data/DIV2K_train_HR.zip -d data
!unzip -q ./data/DIV2K_valid_HR.zip -d data
!rm ./data/DIV2K_train_HR.zip
!rm ./data/DIV2K_valid_HR.zip

In [ ]:
!python -c "import torch; print(torch.__version__)"

In [ ]:
!python -c "import torchvision; print(torchvision.__version__)"

In [ ]:
!pip install torchvision==0.7

In [ ]:
!pip install torch==1.6

In [ ]:
!pip install cudatoolkit=10.1

In [8]:
# Init Date time
from datetime import datetime
now = datetime.now()

#Set Training Path
traininPaht = "/content/drive/MyDrive/SRGAN/training_results/" + datetime.strftime(now,'%Y-%m-%d_%H:%M:%S') 
!mkdir $traininPaht
traininPaht = traininPaht + '/'

statisticsPaht = "/content/drive/MyDrive/SRGAN/statistics/"+datetime.strftime(now,'%Y-%m-%d_%H:%M:%S')
!mkdir $statisticsPaht
statisticsPaht = statisticsPaht + '/'

In [ ]:
! python train.py \
  --num_epochs 100 \
  --use_cuda 1 \
  --batch_size 1 \
  --snapshots_folder /content/drive/MyDrive/SRGAN/snapshots/ \
  --snapshots_train_data Epoch_4_TrainTimes_240000.pth \
  --train_path /content/SRGAN_Test/data/DIV2K_train_HR/ \
  --valid_path /content/SRGAN_Test/data/DIV2K_valid_HR/ \
  --statistics_path $statisticsPaht/ \
  --epochs_path $traininPaht/ \
  --willy_test 0 \
  --do_resize 1

In [ ]:
!cp -R ./epochs/ ../drive/My\ Drive/

In [ ]:
!cp -R ./training_results ../drive/My\ Drive/

In [ ]:
!cp -R ./training_results ../drive/My\ Drive/

In [11]:
!rm *.bmp